In [1]:
import pandas as pd

from shared_utilities import helpers

In [7]:
df_inference = helpers.get_event_propensity_scoring_dataset(
    cluster="prod-app",
    database="stlrnhljets",
    lkupclientid=92,
    game_date="2022-10-14"
)

df_inference.shape

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

LVGT-NJFJ
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/
Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(199392, 12)

In [9]:
df_inference_2 = helpers.get_event_propensity_scoring_dataset(
    cluster="prod-app",
    database="stlrnhljets",
    lkupclientid=92,
    game_date="2022-10-22"
)

df_inference_2.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(199392, 12)

In [10]:
df_inference_3 = helpers.get_event_propensity_scoring_dataset(
    cluster="prod-app",
    database="stlrnhljets",
    lkupclientid=92,
    game_date="2022-10-24"
)

df_inference_3.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(199392, 12)